## SQLAlchemy database connector

SQLAlchemy supports

|Dialect|Required Packages|
|-----|---|
|[Firebird](https://docs.sqlalchemy.org/en/latest/dialects/firebird.html)| `fdb` |
|[Microsoft SQL Server](https://docs.sqlalchemy.org/en/latest/dialects/mssql.html)| `pyodbc`, `pymssql` |
|[MySQL](https://docs.sqlalchemy.org/en/latest/dialects/mysql.html)| `mysqlclient` |
|[PostgreSQL](https://docs.sqlalchemy.org/en/latest/dialects/postgresql.html) | `psycopg2` |
|[SQLite](https://docs.sqlalchemy.org/en/latest/dialects/sqlite.html)| `sqlite` |
|[Sybase](https://docs.sqlalchemy.org/en/latest/dialects/sybase.html)| `python-sybase` |
|[IBM DB2 and Informix](https://github.com/ibmdb/python-ibmdb) | `ibm_db_sa` |
|[Oracle](https://docs.sqlalchemy.org/en/latest/dialects/oracle.html)| `cx_oracle`, `oracle-instantclient` |
|[HIVE and Presto](https://github.com/dropbox/PyHive) | `pyhive` |
|[Impala](https://github.com/cloudera/impyla) | `impyla`|
|[Snowflake](https://docs.snowflake.net/manuals/user-guide/sqlalchemy.html) | `snowflake-sqlalchemy`, `snowflake-connector-python` |

In order to connect to other database flavors, the protocol will need to be changed along with details of your connection including hostname, username, password, and others.


For example, connecting to an Oracle database may require the following information

```python
username = 'me'
password = 'secret'
db_host = 'db.example.com'
port = 1530
service_id = 'my_sid'

engine = create_engine(f'oracle://{username}:{password}@{db_host}:{port}/{service_id}')
```

For this notebook a SQLite database is provided. The same SQLAlchemy function calls will work on any database flavor. **Note** that not all databases support the same SQL operations.

## Engines and connections

In [ ]:
from sqlalchemy import create_engine

db = create_engine('sqlite:///db.sqlite')

With the engine you can inspect the database. Most commonly this is used to determine the table names.

In [ ]:
db.table_names()

In this example a temporary table called `high_mpg` is created and selected from. Once the context manager finishes the temporary table will no longer be available.

The connection context manager lets you setup multiple execution *transactions*. On exit those transactions are *committed* and executed on the database.

In [ ]:
with db.connect() as connection:
    connection.execute('create temp table high_mpg as select * from autompg where mpg > 30')
    result = connection.execute("select * from high_mpg where origin = 'America'")  
    data = result.fetchall()

The data returned by `.fetchall()` is a Python list.

In [ ]:
for row in data:
    print(f'19{row[-3]} {row[-1]:40s}: {row[1]:4.1f} mpg')

## Pandas DataFrames

Since databases contain tabular data reading directly in a Pandas DataFrame can be very convenient. Queries can be performed directly in `.read_sql()`.

In [ ]:
import pandas as pd

df = pd.read_sql('select * from autompg where yr > 75', db)

Plotting with Pandas DataFrames is easy with [HvPlot](https://hvplot.pyviz.org/).

In [ ]:
import hvplot.pandas

plot = df.hvplot.scatter(x='hp', y='mpg', c='origin',
                         hover_cols=['name','yr'], legend='top_right',
                         width=900, padding=0.02)
plot

The `connection` can also be used to mix SQL statement execution and `.read_sql()`. Again we're making a temporary table and using that to select and read into a Pandas DataFrame.

In [ ]:
with db.connect() as connection:
    connection.execute('create temp table low_mpg as select * from autompg where mpg < 20')
    low_mpg = pd.read_sql("select * from low_mpg where origin = 'Asia'", connection)  

low_mpg

## Dask DataFrame

[Dask](https://dask.org) let's us process large amounts of data in parallel and *out-of-core*. That means that we don't need a large-memory resource profile, but having more cores will speed up the computation.

Here data is not read into memory when `read_sql_table()` is called.

In [ ]:
import dask.dataframe as dd

db_uri = 'sqlite:///db.sqlite'

weather = dd.read_sql_table('weather', db_uri, index_col='Date')

Notice that `Date` does not appear in the column list. It is now on the index, which allows for some very powerful operations, especially for datetime.

In [ ]:
weather.columns

Let's prepare the total monthly snowfall and monthly average temperature.

In [ ]:
snowfall = weather['Snowfall'].resample('M').sum()
avg_prec = weather['Precipitation Water Equiv'].resample('M').mean()

In order to perform the out-of-core computation we must use `.compute()`. This returns and *in-memory* Pandas object.

**Caution**: Don't compute the *entire* data set if the return number of rows is very large. Dask is useful because the data does not fit in memory.

In [ ]:
result = snowfall.compute()
result.head()

Here hvplot will perform `.compute()` operations.

In [ ]:
import hvplot.dask

snow_plot = snowfall.hvplot.line(y='Snowfall', title='Total Monthly Snowfall')
prec_plot = avg_prec.hvplot.line(y='Precipitation Water Equiv', title='Average precipitation')

plot = (snow_plot + prec_plot).cols(1)
plot

----

<font color='grey'><i>Copyright Anaconda 2012-2019 All Rights Reserved.</i></font>